# LAB01 - Deteccion Pishing

## Security Data Science
* César Rodas 16776
*


### Librerías

In [13]:
import pandas as pd
import numpy as np
import  matplotlib.pyplot as plt
import seaborn as sns

#!{sys.executable} -m pip install pandas_profiling
import pandas_profiling as pp
from pandas_profiling import ProfileReport

import sklearn
from sklearn import metrics, model_selection, tree

from urllib.parse import urlparse

## Parte 1

## Exploración de datos

In [5]:
# 1. Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones.

#Carga de datos
df = pd.read_csv('dataset_pishing.csv')

In [7]:
# contenido del dataset
df.head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [8]:
#2. Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como “pishing”.

df['status'].value_counts()  

legitimate    5715
phishing      5715
Name: status, dtype: int64

**¿Está balanceado el dataset?**

Sí, porque en la columna [status] existe el mismo numero de registros para _legitimate_ y _pishing_

## Derivación de características

##### f1 - Full url length

In [10]:
df['url_length'] = df['url'].str.len()

##### f2 - Hostname length

In [15]:
def getHostname(url):
  hostname = urlparse(url).netloc
  return hostname

df['hostname'] = df['url'].apply(getHostname)

In [16]:
df['hostname']

0                                      www.crestonwood.com
1                                  shadetreetechnology.com
2        support-appleld.com.secureupdate.duilawyeryork...
3                                              rgipt.ac.in
4                                          www.iracing.com
                               ...                        
11425                                    www.fontspace.com
11426                                   www.budgetbots.com
11427                                     www.facebook.com
11428                       www.mypublicdomainpictures.com
11429                                       174.139.46.123
Name: hostname, Length: 11430, dtype: object

In [17]:
df['hostname_length'] = df['hostname'].str.len()

##### f4-20 - Special characters

In [18]:
# '.' (f4), '-' (f5), '@' (f6) , '?' (f7), '&' (f8), '|' (f9), '=' (f10), '_' (f11), '˜' (f12),
# '%' (f13), '/' (f14), '*' (f15), ':' (f16), ',' (f17), ';' (f18), '$' (f19), '%20' or space (f20)

def number_of_occurrences(url):
  caracteres = url.count('.') + url.count('-') + url.count('@') + url.count('?') + url.count('&') + url.count('|') + url.count('=') + url.count('_') + url.count('˜') + url.count('%') + url.count('/') + url.count('*') + url.count(':') + url.count(',') + url.count(';') + url.count('$') + url.count('%20') + url.count(' ') 
  return caracteres

df['special_characters_count'] = df['url'].apply(number_of_occurrences)  

##### f25 - Use HTTPS

In [21]:
def getProtocol(url):
  protocol = urlparse(url).scheme 
  return protocol

df['protocol'] = df['url'].apply(getProtocol)


In [23]:
df['protocol'].value_counts()

http     6983
https    4447
Name: protocol, dtype: int64

In [26]:
df['use_https'] = df['protocol']

df['use_https'] = df['use_https'].replace(to_replace='https', value=1)
df['use_https'] = df['use_https'].replace(to_replace='http', value=0)

##### f26 - Ratio of digits in full URLs

In [35]:
df['url_digits_count'] = df['url'].str.count('[0-9]')
# df['url_length'] = df['url'].str.len()
df['ratio_digits_url'] = df['url_digits_count'] / df['url_length']

##### f27 - Ratio of digits in full Hostname

In [30]:
df['hostname_digits_count'] = df['hostname'].str.count('[0-9]')

df['ratio_digits_hostname'] = df['hostname_digits_count'] / df['hostname_length']

**Ejemplo del dataset con las columnas nuevas**

In [45]:
df.iloc[:, [67, 68, 69, 70, 71, 72, 73, 74, 75, 76 ]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   url_length                11430 non-null  int64  
 1   hostname                  11430 non-null  object 
 2   hostname_length           11430 non-null  int64  
 3   special_characters_count  11430 non-null  int64  
 4   protocol                  11430 non-null  object 
 5   use_https                 11430 non-null  int64  
 6   url_digits_count          11430 non-null  int64  
 7   ratio_digits_url          11430 non-null  float64
 8   hostname_digits_count     11430 non-null  int64  
 9   ratio_digits_hostname     11430 non-null  float64
dtypes: float64(2), int64(6), object(2)
memory usage: 893.1+ KB


In [48]:
df.iloc[:, [0, 67, 69, 70, 72, 74, 76 ]].head()

,url,url_length,hostname_length,special_characters_count,use_https,ratio_digits_url,ratio_digits_hostname
0,http://www.crestonwood.com/router.php,37,19,7,0,0.000000,0.0
1,http://shadetreetechnology.com/V4/validation/a...,77,23,7,0,0.220779,0.0
2,https://support-appleld.com.secureupdate.duila...,126,50,19,1,0.150794,0.0
3,http://rgipt.ac.in,18,11,5,0,0.000000,0.0
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,10,0,0.000000,0.0


In [49]:
#eliminar las columnas que fueron creadas para la derivación de caracteristicas y no son necesarias

df.drop(['hostname'], axis = 1, inplace = True)
df.drop(['protocol'], axis = 1, inplace = True)
df.drop(['url_digits_count'], axis = 1, inplace = True)
df.drop(['hostname_digits_count'], axis = 1, inplace = True)

## Preprosesamiento

In [51]:
# convertir status a variable binaria
df['status'].value_counts()

legitimate    5715
phishing      5715
Name: status, dtype: int64

In [52]:
df['status'] = df['status'].replace(to_replace='legitimate', value=1)
df['status'] = df['status'].replace(to_replace='phishing', value=0)

In [53]:
# eliminar la columna del dominio

df.drop(['url'], axis = 1, inplace = True)

## Visualización de resultados

In [54]:
# reporte con pandas profiling
pd_profile = ProfileReport(df, title='Visualización de resultados')
pd_profile.to_file('Reporte_de_perfil.html')

Summarize dataset:  99%|█████████▉| 1168/1174 [03:36<00:01,  5.20it/s, Missing diagram bar]                                          C:\Users\cesarvin\AppData\Local\Programs\Python\Python39\lib\site-packages\missingno\missingno.py:250: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right', fontsize=fontsize)
C:\Users\cesarvin\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas_profiling\model\missing.py:89: UserWarning: There was an attempt to generate the Count missing values diagrams, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(missing_diagrams={"Count": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'The number of FixedLocator locations (7), usually from a call to set_ticks, does not match the number of ticklabels (7